# Keras Model Building

In the last flow, we need to deploy the model to Sagemaker. However, SageMaker is easier to use with one of the pre-defined model types - in this case Tensorflow. 

In fact, our deployment strategy for the KNN-based model we trained is to first "export" it to a TF-Recs model with keras (the function keras_model), and then deploy it to SageMaker with their TensorFlowModel abstraction. 

First we import the packages we need and define some config variables:

In [ ]:
import os

for _ in range(3):
    if os.path.exists(f'{os.getcwd()}/setup.py'):
        break
    os.chdir('..')
print('Current working directory:', os.getcwd())

In [ ]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'

In [ ]:
import random
import tarfile
import time
from pathlib import Path
from typing import Tuple

import numpy as np
import tensorflow as tf
import tensorflow_recommenders as tfrs

from src.utils.logging import bprint
from src.utils.meta import get_latest_successful_run
from src.utils.styling import apply_styling

In [ ]:
colors = apply_styling()
palette = colors['palette']

Let's retrieved the artifacts from the latest successful run. 
The `get_latest_successful_run` uses the `metaflow.Flow` object to get results of runs using the (class) name of the flows. 

In [ ]:
FLOW_NAME = 'ModelingFlow'
latest_run = get_latest_successful_run(FLOW_NAME)
final_vectors = latest_run.data.final_vectors
final_dataset = latest_run.data.final_dataset

In [ ]:
type(latest_run)

### Retrieval Model

First, we gather the ids and the embeddings of the songs

In [ ]:
songs_ids = np.array(final_vectors.index_to_key)
songs_embeddings = np.array([final_vectors[idx] for idx in songs_ids])

We need to include an "unknown" item in the embedding matrix.

In [ ]:
num_embeddings = len(songs_embeddings)
embedding_size = songs_embeddings[0].shape[0]
bprint(f'Num of embeddings: {len(songs_embeddings):,}', level=3)
bprint(f'Embeddings dimensions: {embedding_size}', level=3)

bprint('Adding a vector for unknown items', level=3)
unknown_vector = np.zeros((1, embedding_size))
embedding_matrix = np.vstack([unknown_vector, songs_embeddings])
bprint('First item:', embedding_matrix[0][0:5], level=4)
bprint('Shape of the matrix:', embedding_matrix.shape, level=4)
assert embedding_matrix[0][0] == 0.0

bprint('Initializing layers and network', level=3)
lookup_layer = tf.keras.layers.StringLookup(vocabulary=songs_ids, mask_token=None)
embedding_layer = tf.keras.layers.Embedding(
    input_dim=embedding_matrix.shape[0],
    output_dim=embedding_matrix.shape[1],
    weights=[embedding_matrix],
    trainable=False,
)
embedding_layer.build((None,))

model = tf.keras.Sequential([lookup_layer, embedding_layer])

bprint('Creating retrieval model', level=3)
brute_force = tfrs.layers.factorized_top_k.BruteForce(model)
song_index = brute_force.index(candidates=songs_embeddings, identifiers=songs_ids)

Finally, we test the model with one of the songs.

In [ ]:
def get_recommendations(song_id, k=10):
    song_vector = model(np.array([song_id]))
    rec_scores, rec_ids = song_index(tf.constant([song_id]), k=k)
    return song_vector, rec_scores, rec_ids

def pprint_recommendations(song_id, k=10):
    song_vector, rec_scores, rec_ids = get_recommendations(song_id, k=k)
    bprint('Song ID:', song_id, level=4, prefix='*')
    bprint('Song vector:', song_vector.numpy()[0][:5], level=4)
    bprint('Recommendations after track:', level=4)
    for score, song_id in zip(rec_scores[0].numpy(), rec_ids[0].numpy()):
        song_id = str(song_id, 'utf-8')
        if not song_id == song_id:
            continue
        bprint(f'{score:.2f} - {song_id}', level=5)

In [ ]:
type(get_recommendations('Alabimbombao~', k=5)[1])

In [ ]:
bprint('Testing retrieval model', level=3)
test_id = 'Alabimbombao~'  # Unknown!
pprint_recommendations(test_id, k=5)

test_index = 3
test_id = songs_ids[test_index]
pprint_recommendations(test_id, k=5)